In [ ]:
import pandas as pd
import uuid
from code.psql_utils import get_connection, config

In [ ]:
# Data read
filepath = "../data/topic_modeling/intelligence_artificielle/doc_info_sentiments.csv"
question = "C’est la dernière question : souhaitez-vous nous partager d’autres remarques ou idées dans cette consultation sur l’intelligence artificielle ? ,L’intelligence artificielle : quelles priorités ?"
consultation_name = "intelligence_artificielle"
doc_infos = pd.read_csv(filepath, index_col=0)

In [ ]:
def prep_before_sql(doc_infos: pd.DataFrame)-> pd.DataFrame:
    preped_df = doc_infos.copy()
    columns = {"Document": "text",
               "Probability": "topic_probability",
               "Topic": "topic"}
    preped_df = preped_df.rename(columns=columns)
    preped_df.loc[:, "sub_topic_id"] = preped_df.groupby(["topic", "sub_topic"]).topic.transform(lambda g: uuid.uuid4())
    preped_df.loc[:, "topic_id"] = preped_df.groupby("topic").topic.transform(lambda g: uuid.uuid4())
    
    #Here the column taken before apply ["topic"] doesn't matter, it is just to generate a random uuid for each row
    preped_df.loc[:, "id"] = preped_df["topic"].apply(lambda g: uuid.uuid4())
    
    origin_response = preped_df.groupby("old_index").agg(origin_response_id=("id", "first"))
    preped_df = preped_df.join(origin_response, on="old_index")
    #preped_df["fracking_count"] = preped_df["fracking_count"].astype(int)

    columns_to_keep = ["id", "text", "topic_id", "sub_topic_id", "origin_response_id", "topic_probability", "sentiment", "sentiment_score", "Name", "sub_name", "fracking_count", "Representative_document"]
    return preped_df[columns_to_keep]

In [ ]:
doc_infos

In [ ]:
def prep_sub_topics(preped_df: pd.DataFrame):
    sub_topics = preped_df.groupby("sub_topic_id").agg(name=("sub_name", "first"), parent_topic_id=("topic_id", "first")).reset_index()
    sub_topics = sub_topics.rename(columns={"sub_topic_id": "id"})
    return sub_topics

def prep_topic(preped_df: pd.DataFrame, question: str, consultation_name: str)-> pd.DataFrame:
    topics = preped_df.groupby("topic_id").agg(name=("Name", "first")).reset_index()
    topics = topics.rename(columns={"topic_id": "id"})
    sub_topics = prep_sub_topics(preped_df)
    
    consultation_id = uuid.uuid4()
    consultation_df = pd.DataFrame({"title": [consultation_name], "id":[consultation_id]})
    question_id = uuid.uuid4()
    question_df = pd.DataFrame({"title": [question], "id":[question_id], "consultation_id": [consultation_id]})
    topics_df = pd.concat([topics, sub_topics])
    topics_df["question_id"] = question_id
    return consultation_df, question_df, topics_df

# CREATE TABLE representative_responses (
#     response_id UUID NOT NULL,
#     topic_id UUID NOT NULL,
#     FOREIGN KEY (response_id) REFERENCES responses(id),
#     FOREIGN KEY (topic_id) REFERENCES topics(id)
# );


def prep_representative_answers(preped_df: pd.DataFrame):
    representative_df = preped_df[preped_df["Representative_document"]][["topic_id", "id"]]
    representative_df = representative_df.rename(columns={"id": "response_id"})
    return representative_df

In [ ]:
preped_df = prep_before_sql(doc_infos)
representative_df = prep_representative_answers(preped_df)
#preped_df

In [ ]:
consultation_df, question_df, topics_df = prep_topic(preped_df, question, consultation_name)
responses_df = preped_df.drop(columns=["Name", "sub_name", "Representative_document"])
topics_df

## Insertion

In [ ]:
from sqlalchemy import create_engine, select, inspect
from sqlalchemy.engine import URL

def get_engine(section="postgresql"):
    params = config(section=section)
    url_object = URL.create("postgresql", **params)
    engine = create_engine(url_object)
    return engine

In [ ]:
engine = get_engine(section="agora_nlp_alc")

In [ ]:
inspector = inspect(engine)

In [ ]:
inspector.get_schema_names()

In [ ]:
inspector.get_table_names("public")

In [ ]:
#inspector.get_columns("topics")

In [ ]:
connection = engine.connect()

In [ ]:
if_exists="append"
consultation_df.to_sql("consultations", connection, if_exists=if_exists, index=False)

In [ ]:
question_df.to_sql("questions", connection, if_exists=if_exists, index=False)

In [ ]:
topics_df.to_sql("topics", connection, if_exists=if_exists, index=False)

In [ ]:
schema = inspector.get_columns("responses")
expected_cols = [col.get("name") for col in schema]
expected_cols_type = [col.get("type") for col in schema]
print(expected_cols)
print(expected_cols_type)

In [ ]:
actual_cols = responses_df.columns.values
for col in expected_cols:
    #print(expected_cols)
    if col not in actual_cols:
        print(col)

In [ ]:
responses_df.to_sql("responses", connection, if_exists=if_exists, index=False)

In [ ]:
representative_df.to_sql("representative_responses", connection, if_exists=if_exists, index=False)

In [ ]:
responses_df

## Vérification

In [ ]:
def drop_cascade(table: str, conn):
    cursor = conn.cursor()
    result = cursor.execute(f"DELETE FROM {table};")
    return result

In [ ]:
agora_nlp_conn = get_connection(section="agora_nlp_psy")

In [ ]:
connection = engine.connect()

In [ ]:
drop_cascade("consultations", connection)

In [ ]:
pd.read_sql_query("SELECT * FROM questions", connection)

In [ ]:
conn = get_connection("local_prod")

In [ ]:
pd.read_sql_query("""SELECT *
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = N'consultations'""", conn)

In [ ]:
id = "5563aeda-092a-11ee-be56-0242ac120002"
df = pd.read_sql_query(f"SELECT * FROM reponses_consultation WHERE question_id='{id}'", con=conn)
df

In [ ]:
cursor = conn.cursor()

In [ ]:
stmt = """SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'"""
cursor.execute(stmt)
result = cursor.fetchall()

In [ ]:
result

In [ ]:
query = "SELECT * FROM questions"
cursor.execute(query)
result = cursor.fetchall()
result

In [ ]:
questions = pd.read_sql_query("SELECT * FROM questions WHERE type='open'", con=conn)
questions

In [ ]:
result

In [ ]:
stmt = """SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'"""

In [ ]:
cursor.execute(stmt)

In [ ]:
test = pd.read_sql_query('select * from public.topics',con=engine)
test

In [ ]:
test

## Read Questions

In [ ]:
conn = get_connection(section="agora_nlp")

In [ ]:
result = pd.read_sql("SELECT * FROM questions", conn)
result

In [ ]:
id = "2c154084-0929-11ee-be56-0242ac120002"
reponses_multiples = pd.read_sql_query(f"SELECT * from reponses_consultation WHERE question_id='{id}'", conn)
reponses_multiples[reponses_multiples["response_text"] != ""]

In [ ]:
import pandas as pd
query = f"SELECT * FROM responses_qag WHERE type='open'"
pd.read_sql_query(query, conn)

In [ ]:
result

TESTS

In [ ]:
from code.data_preparation import prep_answer_df
import pandas as pd

In [ ]:
df = pd.read_parquet("data/cleaned_data.parquet")
df

In [ ]:
from code.data_preparation import fracking, compute_response_size
import re

def prep_answer_df(df: pd.DataFrame, response_col: str):
    cleaned_df = df.copy()
    NUMBERED_LIST_HANDLER = lambda x: re.sub(r'([0-9]+\. )', ' ', x)
    SPECIAL_CHAR_HANDLER = lambda x: re.sub(r'[()/\\]', ' ', x)
    WHITESPACE_HANDLER = lambda x: re.sub('\s+', ' ', re.sub('\n+', ' ', x.strip()))
    cleaned_df["cleaned_text"] = cleaned_df[response_col].apply(NUMBERED_LIST_HANDLER)
    cleaned_df["cleaned_text"] = cleaned_df["cleaned_text"].apply(SPECIAL_CHAR_HANDLER)
    cleaned_df["cleaned_text"] = cleaned_df["cleaned_text"].apply(WHITESPACE_HANDLER)
    print(cleaned_df)
    fracked_df = fracking(cleaned_df, "cleaned_text", sep=".")
    fracked_df = fracked_df.dropna(axis=0, subset=["fracked_text"])
    fracked_df = compute_response_size(fracked_df, "fracked_text")
    return fracked_df